In [1]:
import sys
sys.path.append('..')

import login
login.connect()
from hheise_scripts import hheise_util as util
from schema import hheise_behav, common_exp, common_mice, common_img, hheise_placecell

import matplotlib
%matplotlib qt

Connecting hheise@130.60.53.48:3306


## Enter new Session

The database has to know which sessions exist. This is done by **manual** entries in the `common_exp.Session` table. All other analysis steps descend from here. 
If the session was an imaging session, a manual entry to `common_img.Scan` is also required.
For Hendriks session and data structure, the function `add_many_sessions()` adds sessions for multiple mice on the same day automatically (different tasks and switches have to be provided). Also `common_img.Scan` will be filled with Hendriks default values for imaging sessions, the criterion is **any** TIFF-files being present in the session directory.

In [ ]:
# Add multiple entries to common_exp.Session and common_img.Scan
# Trial switch is given as base 1!
date = '2021-08-15'
mice = (83,89,
        91,94,95)
setup = 'VR_2p'
# task = ['Active', 'Active',
#         'Active', 'Active', 'Active']
# switch = [[-1], [-1],
#           [-1], [8], [-1], [-1]]

# util.add_many_sessions(date=date, mice=mice, setup=setup, task=task, switch=switch)
util.add_many_sessions(date=date, mice=mice, setup=setup)

## VR Behavior analysis

First, `hheise_behav.VRSessionInfo` and `hheise_behav.RawBehaviorFile` have to be populated.
This is because `RawBehaviorFile` pre-screens sessions for possible bad trials, which should be removed manually, including the associated TIFF files. Thus, the TIFF files should not be in the database yet.

In [ ]:
# Make entries for hheise_behav.VRSessionInfo and 
hheise_behav.VRSessionInfo().populate('username="hheise"')

In [ ]:
%matplotlib qt
# Pre-screen trials for bad behavior data while populating hheise_behav.RawBehaviorFile
# -> Does not work in Jupyter Notebook, call in PyCharm console
hheise_behav.RawBehaviorFile().populate()
# Delete TIFF files of bad trials if necessary

## RawImagingFile

Now, `common_img.RawImagingFile` has to be filled before we continue with the VR behavior analysis, because the behavior alignment script needs to know the frame numbers of each trial, which is stored in `common_img.RawImagingFile`.

In [2]:
# Find and load the raw TIFF files
common_img.RawImagingFile.populate(dict(username='hheise'))

Finding raw imaging files for entry {'username': 'hheise', 'mouse_id': 33, 'day': datetime.date(2020, 8, 18), 'session_num': 1}


ImportError: No files found in W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M33\20200818 that fit the patterns ['file_000??.tif', 'f_000??.tif']!

## Finish VR behavior pipeline

Now we can populate the rest of the VR behavior tables without interruption.

In [ ]:
%%notify

# Analyse the VR behavior, now that frame counts of imaging trials are loaded in the database
hheise_behav.VRLogFile.populate(display_progress=True)
hheise_behav.VRLog.populate(display_progress=True)
hheise_behav.VRSession.populate(display_progress=True)
hheise_behav.VRPerformance.populate(display_progress=True)
hheise_behav.PerformanceTrend.populate(display_progress=True)

## CaImAn pipeline

Here, the actual CaImAn segmentation pipeline starts. Each of these tables should run automatically, but might throw errors, so check in regularly. Computation times are now really long, especially for `MotionCorrection` and `Segmentation` (approx 1h each per session).

In [12]:
%%notify

# After VR behavior is fully analysed, we can start with the CaImAn pipeline
common_img.ScanInfo.populate(dict(username='hheise'), display_progress=True)

ScanInfo:   0%|                                                                                 | 0/41 [00:00<?, ?it/s]

Populating ScanInfo for key: {'username': 'hheise', 'mouse_id': 33, 'day': datetime.date(2020, 8, 18), 'session_num': 1}


ScanInfo:   2%|█▊                                                                       | 1/41 [00:03<02:13,  3.33s/it]

Populating ScanInfo for key: {'username': 'hheise', 'mouse_id': 33, 'day': datetime.date(2020, 8, 19), 'session_num': 1}


ScanInfo:   5%|███▌                                                                     | 2/41 [00:07<02:24,  3.70s/it]

...

Populating ScanInfo for key: {'username': 'hheise', 'mouse_id': 41, 'day': datetime.date(2020, 9, 8), 'session_num': 1}


ScanInfo:  98%|██████████████████████████████████████████████████████████████████████▏ | 40/41 [06:06<00:04,  4.09s/it]

Populating ScanInfo for key: {'username': 'hheise', 'mouse_id': 41, 'day': datetime.date(2020, 9, 11), 'session_num': 1}


ScanInfo: 100%|████████████████████████████████████████████████████████████████████████| 41/41 [06:09<00:00,  9.00s/it]


<IPython.core.display.Javascript object>

In [7]:
# Check which available MotionParameter set you want to use for you motion correction
common_img.MotionParameter()

"motion_id index for unique parameter set, base 0",motion_shortname Short meaningful name of parameter set,motion_description Longer description of conditions for which this parameter set fits,crop_left Pixels to crop on the left to remove scanning artifacts before MC.,"crop_right See crop_left. The actual movie is not cropped here, but in MemoryMappedFile(),",offset Fixed value that is added to all pixels to make mean pixel values positive.,max_shift maximum allowed rigid shifts (in um),"stride_mc stride size for non-rigid correction (in um), patch size is stride+overlap)","overlap_mc Overlap between patches, in pixels","pw_rigid flag for performing rigid or piecewise (patch-wise) rigid mc (0: rigid, 1: pw)",max_dev_rigid maximum deviation allowed for patches with respect to rigid shift,"border_nan flag for allowing NaN in the boundaries. If False, value of the nearest data point",n_iter_rig Number of iterations for rigid motion correction (not used for pw-rigid),nonneg_movie flag for producing a non-negative movie
0,default,Default values that work well from Hendriks previous experience. Uses piecewise-rigid correction.,12,12,220,50,150,30,1,3,0,2,1


In [ ]:
%%notify
# Run motion correction and quality control (which checks quality of motion correction)
common_img.MotionCorrection.populate(dict(username='hheise', motion_id=0), display_progress=True)


In [ ]:
%%notify
common_img.QualityControl.populate(dict(username='hheise'), display_progress=True)

In [ ]:
%%notify
common_img.Segmentation.populate(dict(username='hheise'), display_progress=True, make_kwargs=dict(save_overviews=True))

In [4]:
# Validate Segmentation to spot irregularities in number of accepted components, which might require parameter adaptation
util.validate_segmentation(thr=20, plot_all=True)

Validating sessions of mouse 82...
	 On day 2021-07-16, 20% fewer ROIs than on the previous session were found. (309 vs 541).
Validating sessions of mouse 83...
	 On day 2021-07-24, 20% more ROIs than on the previous session were found. (661 vs 485).
	 On day 2021-08-02, 20% more ROIs than on the previous session were found. (692 vs 570).
Validating sessions of mouse 85...
	 On day 2021-07-09, 20% more ROIs than on the previous session were found. (687 vs 537).
	 On day 2021-07-11, 20% fewer ROIs than on the previous session were found. (423 vs 653).
	 On day 2021-07-14, 20% more ROIs than on the previous session were found. (524 vs 423).
	 On day 2021-07-19, 20% more ROIs than on the previous session were found. (653 vs 424).
	 On day 2021-08-02, 20% fewer ROIs than on the previous session were found. (491 vs 618).
	 On day 2021-08-05, 20% more ROIs than on the previous session were found. (699 vs 491).
	 On day 2021-08-11, 20% fewer ROIs than on the previous session were found. (444 

In [ ]:
(common_img.Segmentation.ROI & 'mouse_id=83')

In [ ]:
# Use different parameter set on selected sessions
session_keys = [dict(username='hheise', mouse_id=82, day='2021-07-16', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=83, day='2021-08-02', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=86, day='2021-07-08', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=86, day='2021-07-21', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=90, day='2021-07-16', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=90, day='2021-07-30', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=91, day='2021-07-20', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=91, day='2021-08-02', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=92, day='2021-07-19', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=93, day='2021-07-08', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=93, day='2021-07-09', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=93, day='2021-07-10', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=94, day='2021-07-08', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=94, day='2021-07-09', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=94, day='2021-07-10', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=94, day='2021-07-11', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=94, day='2021-07-15', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=95, day='2021-07-08', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=95, day='2021-07-09', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=95, day='2021-07-10', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=95, day='2021-07-14', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=95, day='2021-07-19', session_num=1, caiman_id=1),
               dict(username='hheise', mouse_id=95, day='2021-07-20', session_num=1, caiman_id=1)]

# Delete old entries with ID 0 before making new one
for key in session_keys:
    k = key.copy()
    k['caiman_id'] = 0
    (common_img.Segmentation & k).delete()

# Make new entries
for key in session_keys:
    common_img.Segmentation.populate(key)

## Place Cell Analysis

Place Cell Analysis should be straight-forward as well, all tables can be populated in-order.

In [ ]:
# Check entries of PlaceCellParameter and add a parameter set if necessary
hheise_placecell.PlaceCellParameter()

In [ ]:
# Run the pipeline sequentially
# The restriction to my username is only necessary for the first table, since the other tables depend on PCAnalysis
hheise_placecell.PCAnalysis.populate(dict(username='hheise'), display_progress=True)
hheise_placecell.TransientOnly.populate(display_progress=True)
hheise_placecell.Synchronization.populate(display_progress=True)
hheise_placecell.BinnedActivity.populate(display_progress=True)
hheise_placecell.PlaceCell.populate(display_progress=True)

In [ ]:
from importlib import reload
reload(common_img)